In [58]:
import advent

def parse_line(line):
    words = line.split(' ')
    return tuple([int(words[ix]) for ix in [6, 12, 18, 21, 27, 30]])

lines = advent.get_lines(19, map_fn=parse_line)

In [59]:
tadd3 = lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2])
tadd4 = lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2], a[3] + b[3])
tmul = lambda c, a: (c * a[0], c*a[1], c*a[2], c*a[3])

def build_and_mine(blueprint, resources, miners, current_miners):
    # Checks if a suggested number of miners is possible to build with the given resources
    # and if so, return tuple with resources left after building
    ore_needed = (miners[0] * blueprint[0]) + (miners[1] * blueprint[1]) + (miners[2] * blueprint[2]) + (miners[3] * blueprint[4])
    clay_needed = (miners[2] * blueprint[3])
    obsidian_needed = (miners[3] * blueprint[5])
    if ore_needed > resources[0] or clay_needed > resources[1] or obsidian_needed > resources[2]:
        return False, False
    new_miners = tadd4(miners, current_miners)
    new_resources = tadd4(resources, tmul(-1, (ore_needed, clay_needed, obsidian_needed, 0)))
    new_resources = tadd4(new_resources, current_miners)
    return new_resources, new_miners

In [62]:
# Attempt 2: choose the next robot to build, from the list of 4. Then build that next robot ASAP, before generating another robot
# hopefully, we will build maximum like 6 robots, so this will generate 4^6 = 4096 paths, which seems managable
# and many of those paths will be cut off early, such as any path that starts with 'geode' (because you don't have obsidian yet)
# unfortunately in practice we build way more robots than just 6

def amax(*args):
    return max(args, key=lambda x: x[0])

def maximum_geodes_2(blueprint, resources=(0,0,0,0), miners=(1,0,0,0), minute=1, next_build=None, max_minute=24):
    # max_build = 0: ore, 1: clay, 2: obsidian, 3: geode
    if minute == max_minute: return miners[3] + resources[3], []
    if next_build == 2 and miners[1] == 0: return 0, []
    if next_build == 3 and miners[2] == 0: return 0, []
    if miners[0] > 15: return -1, [] # maximum 10 ore miners allowed
    if miners[1] > 15: return -1, [] # maximum 10 clay miners allowed

    if next_build is None:
        return amax(
            maximum_geodes_2(blueprint, resources, miners, minute, 0, max_minute),
            maximum_geodes_2(blueprint, resources, miners, minute, 1, max_minute)
        )

    to_build_tmp = (int(next_build==0), int(next_build==1), int(next_build==2), int(next_build==3))
    new_resources, new_miners = build_and_mine(blueprint, resources, to_build_tmp, miners)
    #print(resources, miners, minute, resources_left, to_build)
    if new_resources:
        next_build_options = {0: [0, 1], 1:[0, 1, 2], 2: [1, 2, 3], 3: [2, 3]}[next_build]
        res = max([maximum_geodes_2(blueprint, new_resources, new_miners, minute+1, i, max_minute) for i in next_build_options], key=lambda x: x[0])
        return res[0], [(new_miners, new_resources, minute+1)] + res[1]
    else:
        # pass the tim
        new_resources = tadd4(resources, miners)
        max_geodes, subresult = maximum_geodes_2(blueprint, new_resources, miners, minute + 1, next_build, max_minute)
        return max_geodes, [(miners, new_resources, minute+1)] + subresult

blueprint1 = (4, 2, 3, 14, 2, 7)
blueprint2 = (2, 3, 3, 8, 3, 12)
m, p = maximum_geodes_2(blueprint1, next_build=None, max_minute=24)

m

9

In [63]:
from tqdm.notebook import tqdm
result = 0

for i, line in tqdm(list(enumerate(lines))):
    result += maximum_geodes_2(line, next_build=None, max_minute=24)[0] * (i+1)

print(result)

  0%|          | 0/30 [00:00<?, ?it/s]

1264


In [64]:
maximum_geodes_2(lines[0], next_build=None, max_minute=32)

(11,
 [((1, 0, 0, 0), (1, 0, 0, 0), 2),
  ((1, 0, 0, 0), (2, 0, 0, 0), 3),
  ((1, 0, 0, 0), (3, 0, 0, 0), 4),
  ((1, 0, 0, 0), (4, 0, 0, 0), 5),
  ((2, 0, 0, 0), (1, 0, 0, 0), 6),
  ((2, 0, 0, 0), (3, 0, 0, 0), 7),
  ((2, 0, 0, 0), (5, 0, 0, 0), 8),
  ((3, 0, 0, 0), (3, 0, 0, 0), 9),
  ((3, 0, 0, 0), (6, 0, 0, 0), 10),
  ((3, 1, 0, 0), (5, 0, 0, 0), 11),
  ((4, 1, 0, 0), (4, 1, 0, 0), 12),
  ((4, 2, 0, 0), (4, 2, 0, 0), 13),
  ((4, 3, 0, 0), (4, 4, 0, 0), 14),
  ((4, 4, 0, 0), (4, 7, 0, 0), 15),
  ((4, 5, 0, 0), (4, 11, 0, 0), 16),
  ((4, 6, 0, 0), (4, 16, 0, 0), 17),
  ((4, 6, 1, 0), (4, 8, 0, 0), 18),
  ((4, 7, 1, 0), (4, 14, 1, 0), 19),
  ((4, 7, 2, 0), (4, 7, 2, 0), 20),
  ((4, 8, 2, 0), (4, 14, 4, 0), 21),
  ((4, 8, 3, 0), (4, 8, 6, 0), 22),
  ((4, 9, 3, 0), (4, 16, 9, 0), 23),
  ((4, 9, 4, 0), (4, 11, 12, 0), 24),
  ((4, 9, 4, 0), (8, 20, 16, 0), 25),
  ((4, 9, 4, 1), (9, 29, 4, 0), 26),
  ((4, 9, 5, 1), (9, 24, 8, 1), 27),
  ((4, 9, 6, 1), (9, 19, 13, 2), 28),
  ((4, 9, 7, 1), (

In [65]:
maximum_geodes_2(lines[1], next_build=None, max_minute=32)

(47,
 [((1, 0, 0, 0), (1, 0, 0, 0), 2),
  ((1, 0, 0, 0), (2, 0, 0, 0), 3),
  ((1, 0, 0, 0), (3, 0, 0, 0), 4),
  ((1, 0, 0, 0), (4, 0, 0, 0), 5),
  ((2, 0, 0, 0), (1, 0, 0, 0), 6),
  ((2, 0, 0, 0), (3, 0, 0, 0), 7),
  ((2, 0, 0, 0), (5, 0, 0, 0), 8),
  ((3, 0, 0, 0), (3, 0, 0, 0), 9),
  ((3, 1, 0, 0), (3, 0, 0, 0), 10),
  ((3, 2, 0, 0), (3, 1, 0, 0), 11),
  ((3, 3, 0, 0), (3, 3, 0, 0), 12),
  ((3, 4, 0, 0), (3, 6, 0, 0), 13),
  ((3, 5, 0, 0), (3, 10, 0, 0), 14),
  ((3, 6, 0, 0), (3, 15, 0, 0), 15),
  ((3, 6, 1, 0), (3, 10, 0, 0), 16),
  ((3, 7, 1, 0), (3, 16, 1, 0), 17),
  ((3, 7, 2, 0), (3, 12, 2, 0), 18),
  ((3, 7, 3, 0), (3, 8, 4, 0), 19),
  ((3, 7, 3, 0), (6, 15, 7, 0), 20),
  ((3, 7, 4, 0), (6, 11, 10, 0), 21),
  ((3, 7, 4, 1), (5, 18, 7, 0), 22),
  ((3, 7, 4, 2), (4, 25, 4, 1), 23),
  ((3, 7, 5, 2), (4, 21, 8, 3), 24),
  ((3, 7, 5, 3), (3, 28, 6, 5), 25),
  ((3, 7, 5, 3), (6, 35, 11, 8), 26),
  ((3, 7, 5, 4), (5, 42, 9, 11), 27),
  ((3, 7, 5, 5), (4, 49, 7, 15), 28),
  ((3, 7, 5, 

In [66]:
maximum_geodes_2(lines[2], next_build=None, max_minute=32)

(25,
 [((1, 0, 0, 0), (1, 0, 0, 0), 2),
  ((1, 0, 0, 0), (2, 0, 0, 0), 3),
  ((1, 0, 0, 0), (3, 0, 0, 0), 4),
  ((1, 0, 0, 0), (4, 0, 0, 0), 5),
  ((2, 0, 0, 0), (1, 0, 0, 0), 6),
  ((2, 0, 0, 0), (3, 0, 0, 0), 7),
  ((2, 0, 0, 0), (5, 0, 0, 0), 8),
  ((3, 0, 0, 0), (3, 0, 0, 0), 9),
  ((3, 0, 0, 0), (6, 0, 0, 0), 10),
  ((3, 1, 0, 0), (5, 0, 0, 0), 11),
  ((4, 1, 0, 0), (4, 1, 0, 0), 12),
  ((4, 2, 0, 0), (4, 2, 0, 0), 13),
  ((4, 3, 0, 0), (4, 4, 0, 0), 14),
  ((4, 4, 0, 0), (4, 7, 0, 0), 15),
  ((4, 5, 0, 0), (4, 11, 0, 0), 16),
  ((4, 6, 0, 0), (4, 16, 0, 0), 17),
  ((4, 6, 1, 0), (5, 8, 0, 0), 18),
  ((4, 7, 1, 0), (5, 14, 1, 0), 19),
  ((4, 7, 2, 0), (6, 7, 2, 0), 20),
  ((4, 8, 2, 0), (6, 14, 4, 0), 21),
  ((4, 8, 3, 0), (7, 8, 6, 0), 22),
  ((4, 9, 3, 0), (7, 16, 9, 0), 23),
  ((4, 9, 4, 0), (8, 11, 12, 0), 24),
  ((4, 9, 4, 1), (9, 20, 8, 0), 25),
  ((4, 9, 4, 2), (10, 29, 4, 1), 26),
  ((4, 9, 5, 2), (11, 24, 8, 3), 27),
  ((4, 9, 5, 3), (12, 33, 5, 5), 28),
  ((4, 9, 6, 3), 

In [55]:
(
    maximum_geodes_2(lines[0], next_build=None, max_minute=32)[0] * 
    maximum_geodes_2(lines[1], next_build=None, max_minute=32)[0] * 
    maximum_geodes_2(lines[2], next_build=None, max_minute=32)[0]
)

11750

In [1]:
11*47*25

12925